In [15]:
!pip list 

Package                Version
---------------------- ---------
absl-py                0.10.0
appdirs                1.4.4
appnope                0.1.0
astunparse             1.6.3
atari-py               0.2.6
attrs                  19.3.0
backcall               0.2.0
bleach                 3.1.5
cachetools             4.1.1
casadi                 3.5.5
certifi                2020.6.20
chardet                3.0.4
cloudpickle            1.6.0
cycler                 0.10.0
decorator              4.4.2
defusedxml             0.6.0
distlib                0.3.1
entrypoints            0.3
filelock               3.0.12
future                 0.18.2
gast                   0.3.3
google-auth            1.22.1
google-auth-oauthlib   0.4.1
google-pasta           0.2.0
grpcio                 1.32.0
gym                    0.17.3
h5py                   2.10.0
httpsig                1.3.0
idna                   2.10
importlib-metadata     1.6.1
ipykernel              5.3.0
ipython                7.15

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

## SQP solver

reference: Numerical Optimization (Jorge Nocedal and Stephen J. Wright), Springer, 2006.

The purpose of this notebook is to visualize **Sequential Quadratic Programming** method both in line search and trust-region frameworks. 

SQP as an effective method for nonlinearly constrained optimization problem solves a sequence of quadratic optimization problems to approximate the solution of the original nonlinear problem. The solving strategies by applying active-set methods are bifold: IQP and EQP methods. The **inequality-constrained QP** (IQP) approach solves an inequality-constrained quadratic problem iteratively to compute a search step and update the estimate of the optimal active set. **Equality constrained QP** method separates the computations of search step and active set estimation by 1. first, generating an estimate of the optimal active set; 2. then, solving an equality-constrained quadratic program to find the search step.

Assumed that we are given an nonlinear optimization problem as following: 
$$
\begin{aligned}
\text{min} &f(x)\\
\text { subject to } c_{i}(x)&=0,  i \in \mathcal{E} \\
c_{i}(x) &\geq 0,  i \in \mathcal{I},
\end{aligned}
$$
where the objective function $f$ and constraint functions are smooth. 
The quadratic program for the given NLP problem could be derived by the Newton-KKT system with the assumption that the constraints Jacobian $A_k$ has full row rank (linearly independent constraints) and the Hessian of the Lagrangian is positive definite (second order sufficient optimality condition). The underlying idea is to replace the original nonlinear problem by the problem minimizing the Lagrangian, and then to make a quadratic approximation to the Lagrangian.

The following quadratic problem is then solved in each iteration to update the step and the Lagrangian mupltiplier: $$
\begin{array}{rl}
\min _{p} & f_{k}+\nabla f_{k}^{T} p+\frac{1}{2} p^{T} \nabla_{x x}^{2} \mathcal{L}_{k} p \\
\text { subject to } & \nabla c_{i}\left(x_{k}\right)^{T} p+c_{i}\left(x_{k}\right)=0, \quad i \in \mathcal{E} \\
& \nabla c_{i}\left(x_{k}\right)^{T} p+c_{i}\left(x_{k}\right) \geq 0, \quad i \in \mathcal{I}.
\end{array}
$$


### Some important ingredients
1. Handling in consistent linearizations: The linearized constriants of the original nonlinear constraints may give rise to an infeasible subproblem. Thus, we could reformulate the NLP as a $\ell_1$ penalty problem.
2. Full Quasi-Newton approximations: BFGS and SR1.
    - BFGS: If Hession $\nabla_{x x}^{2} \mathcal{L}$ has negative eigenvalues, BFGS updating may not fulfilled the curvature condtion $s_{k}^{T} y_{k}>0$. We may skip the update if $s_{k}^{T} y_{k} \geq \theta s_{k}^{T} B_{k} s_{k}$ not satisfied, or, we could replace the $y_{k}$ in the standard BFGS by $r_{k}=\theta_{k} y_{k}+\left(1-\theta_{k}\right) B_{k} s_{k}$.
    - SR1: A Good choice for a trust-region method, but not accept an indefinite Hessian approximation for a Line search method.
3. Reduced-Hessian Quasi-Newton approximations: Find approximations to only the part of $\nabla_{x x}^{2} \mathcal{L}_{k}$ that affects the component of $p_k$ in the null space of $A_k$.
4. Merit fucntions: control the step size in a line search method and determine whether the step should be accepted or rejected in a trust-region method. Many of merit functions could be applied, for example nonsmooth penalty functions and augmented Lagrangians.
5. Second-order correction: To keep away from the Maratos effect by using the new step to approxiamte a second-order Taylor expandsion.


### Line search SQP

Step 1: Initialization: objective function, its first and second derivative (or approximated Hessian instead), constraint Jacobian, and constraint.

Step 2: Solve the approximated squadratic program to get the step and Lagrange multiplier. 

Step 3: select $\mu$ satisfying $\mu \geq \frac{\nabla f_{k}^{T} p_{k}+(\sigma / 2) p_{k}^{T} \nabla_{x x}^{2} \mathcal{L}_{k} p_{k}}{(1-\rho)\left\|c_{k}\right\|_{1}}$ for the merit function.

Step 4: Select $\alpha_k$ satisfying the sufficient decrease condition (Armijo condition).

Step 5: Evaluation and jump to step 2

### Trust-region SQP

`TODO: finish it`

In [3]:
# small number is to be recognized as zero to avoid numerical problem (owning to the calculation accuracy)
tol_zero = 1e-10

In [4]:
def active_set(ai, bi, x0):
    '''
    Initialize the active set for the given initial state.
    Args:
        ai: Left-hand side of the inequality constraints (Ni x Nx).
        bi: right-hand side of the inequality constraints (Ni x 1).
        x0: initial state (Nx x 1).
        
    Return:
        active_set: rerturn the active set of the given inequality constraints at x0 (N_act x Nx).
        inactive_set: rerturn the inactive set of the given inequality constraints at x0 (N_inact x Nx).
    '''
    n_active_row = (ai @ x0 - bi == 0).flatten()
    n_inactive_row = ~ n_active_row
    active_set = np.hstack((ai[n_active_row],bi[n_active_row]))
    inactive_set = np.hstack((ai[n_inactive_row],bi[n_inactive_row]))
    return active_set,inactive_set

In [5]:
def QP_subproblem(G, c, ws, xk):
    
    '''
    Solve the QP subproblem for Newton-KKT system.
    Args:
        G: Weigting matrix of quadratic terms (Nx x Nx).
        c: Weigting matrix of linear terms (Nx x 1).
        ws: working space ( (Ne + N_active) x (Nx + 1)).
        xk: Curret state (Nx x 1).
        
    Return:
        pk: search step (Nx x 1). 
        lambda_k: Lagrangian multiplier ( (Ne + N_active) x 1
    '''
    
    
    Nx = np.shape(xk)[0]
    Nlambda = np.shape(ws)[0]
    
    A = ws[:,0:-1]
    
    gk = G @ xk + c
    rhs = np.vstack((-gk,np.zeros([Nlambda,1])))
    lhs = np.vstack((np.hstack((G,-A.T)),np.hstack((A,np.zeros([Nlambda,Nlambda])))))
    sol = np.linalg.inv(lhs) @ rhs
    sol[abs(sol) < tol_zero] = 0.0
    return sol[:Nx,:],sol[Nx:,:]

In [6]:
def QP_activeset(G, c, ae = None, be = None, ai= None, bi= None, x0 = None):
    '''
    Solve the convex QP problem with active-set method
    Args:
        G: Weigting matrix of quadratic terms (Nx x Nx).
        c: Weigting matrix of linear terms (Nx x 1).
        ae: Left-hand side of the equality constraints (Ne x Nx).
        be: Right-hand side of the equality constraints  (Ne x 1).
        ai: Left-hand side of the inequality constraints (Ni x Nx).
        bi: Right-hand side of the inequality constraints (Ni x 1).        
        x0: Initial state (Nx x 1).    
    
    Returns: 
        xk: Optimial state.
    
    TODO:
        1. deal with infeasible initial point.
        2. Select another algorithm for the QP subproblem.
        3. Tranform into C code.
        4. Visualization.
        5. Find a better way to deal with the empty set.
        6. If G is singular, cannot use the inverse function. -> use Newton-Raphson method instead.
    '''
        
    # Test if the given initial point is feasible.
    Nx = np.shape(G)[0]
    if x0 is None:
        x0 = np.zeros(Nx).reshape(-1,1)
    else:
        x0 = np.reshape(x0,(-1,1))
    if ae is not None:
        if not np.all(ae @ x0 - be == 0):
            print("Error")
            return None
    if ai is not None:
        if not np.all(ai @ x0 - bi >= 0):
            print("Error")
            return None
    xk = x0
    
    #initialization.

    if ae is not None:
        Ne = np.shape(ae)[0]
    else:
        Ne = 0
    if ai is not None:
        Ni = np.shape(ai)[0]
        # Return which set is active. Empty array if none inequality constraint is active.
        ws_active, ws_inactive = active_set(ai, bi, xk)
    else:
        Ni = 0 
        ws_active = np.array([], dtype=np.int).reshape(0,Nx+1)
        ws_inactive = np.array([], dtype=np.int).reshape(0,Nx+1)

    if ae is None:
        we = np.array([], dtype=np.int).reshape(0,Nx+1)
    else:
        we = np.hstack((ae,be))


    # Define working space = equality constraints + active inequality constraints.
    ws = np.vstack((we,ws_active))
    
    while(True):
        # Solve QP subproblem.
        pk,lambda_k = QP_subproblem(G,c,ws,xk)
        # Seperate the Lagrangian multiplier into two parts for equality constraints and inequality constraints. 
        if np.shape(lambda_k)[0] > Ne:
            lambda_inequality = lambda_k[Ne:,:]
        else:
            lambda_inequality = np.array([], dtype=np.int).reshape(0,1)
        n_zero = Nx - np.count_nonzero(pk)
        # If search step vector has only element with 0 value. 
        if n_zero == Nx:
            # p statisfies the optimality conditions, check the signs of the multipliers.
            if np.all(lambda_inequality >= 0):
                print("solution find")
                return xk, lambda_k
            else:
                # Droping the working constraint with most negative value.
                n_min = np.argmin(lambda_inequality)
                row_del = ws_active[n_min,:]
                ws_active = np.delete(ws_active, (n_min), axis=0)
                ws_inactive = np.vstack((ws_inactive, row_del))
                ws = np.vstack((we,ws_active))
        # p vector not zero -> calculate the search length alpha.
        else:
            # Yield alpha value.
            alpha_k = 1
            for i,row in enumerate(ws_inactive):
                b = row[-1]
                a = row[:-1]
                if (a @ pk)[0] < 0:
                    v_tmp = (( b - a @ xk )/ (a @ pk))[0]
                    if v_tmp < 1:
                        n_constraint = i
                        alpha_k = v_tmp
            if alpha_k == 1:
                xk = xk + pk * alpha_k
            # If not a full step, include the most restrive inactive constraint into working space.
            else:
                row_inc = ws_inactive[n_constraint,:]
                ws_active = np.vstack( (ws_active, row_inc))
                ws_inactive = np.delete(ws_inactive, (n_constraint), axis=0)
                ws = np.vstack((we,ws_active))
                xk = xk + pk * alpha_k

### Local SQP 

In [7]:
def sp_sub(var,value):
    if np.ndim(value) == 2:
        value = value.flatten()
    
    return list(zip(var,value))

In [19]:
def SQP_solver(var, obj, lambda_0 = None, x0 = None ,ce = None, ci = None, solver = 'sqp_local', eps = 1e-6):
    '''
    Solve the convex nonlinear problem with SQP (QP subproblem solved by an active-set method)
    Args:
        var: Decision variables in Sympy symbols (Nx x 1).
        obj: Objective function in Sympy containing symbolic variables (1 x 1).
        lambda_0: Initial guess of Lagrangian Multiplier ( (Ne + Ni) x 1).
        x0: Initial state (Nx x 1).
        ce: Equality constraints in sympy Matrix (Ne x 1).
        ci: Inequality constraints in sympy Matrix (Ni x 1)   
        solver: Type of sqp solver. Default local SQP solver.
        eps: Tolerance for convergence test.
    Returns: 
        xk: Optimial decision variable vector.
    TODO: 
    1. Deal with unconstrained problem
    2. Deal with only equality or inequality constraint problem
    3. Line search SQP
    4. Trust region SQP
    '''     
    # Initialization   
    # Decide SQP solver
    if solver is 'sqp_local':
        print("Solving with the local SQP algorithm")

    # Determine the number of decision variables, equality constraints and inequality constraints. 
    Nx = np.shape(var)[0]
    # Test if the given initial point is feasible.
    if x0 is None:
        x0 = np.zeros(Nx).reshape(-1,1)
    if ce is not None:
        if not np.all(np.array(ce.subs(sp_sub(x_vec, x0))) == 0):
            print("Error")
            return None
        Ne = np.shape(ce)[0]
    else:
        Ne = 0
    if ci is not None:
        if not np.all(np.array(ci.subs(sp_sub(x_vec, x0))) >= 0):
            print("Error")
            return None
        Ni = np.shape(ci)[0]
    else:
        Ni = 0
    xk = x0
    # Initialize the Lagrangian Multipliers
    if lambda_0 is None:
        lambda_0 = np.zeros(Ne + Ni).reshape(-1,1)
    lambda_k = lambda_0
    
    # Initialize the objective fn, linearized obj, Hessian, and linearized constraints.
    c_symb = ci.row_insert(0,ce)    #  (Ne + Ni) x 1
    c_jac_symb = c_symb.jacobian(var)    #  (Ne + Ni) x Nx
    lambda_symb = sp.Matrix( sp.symbols('lambda0:' + str(Ne+Ni)))    
    f_jac_symb = obj.jacobian(var).T # column vector    
    L_symb = f_jac_symb - c_jac_symb.T @ lambda_symb
    L_hessian_xx_symb = L_symb.jacobian(var)
    
    while(True):
        # Sensitivity evaluation.
        ck = c_symb.subs(sp_sub(var, xk))
        c_jac_k = c_jac_symb.subs(sp_sub(var, xk))    #  (Ne + Ni) x Nx
        fk = obj.subs(sp_sub(var, xk))
        f_jac_k = f_jac_symb.subs(sp_sub(var, xk))
        L_hessian_xx_k = L_hessian_xx_symb.subs(sp_sub(var, xk)).subs(sp_sub(lambda_symb,lambda_k))

        # Set as float to avoid problem raised in np.linalg.inv.
        # Reconstruct and reshape all the matrix for active-set method QP solver.
#         G = np.array(L_hessian_xx_k,dtype ='float')
#         c = np.array(f_jac_k,dtype ='float')
#         ae = np.array(c_jac_k,dtype ='float')[:Ne,:]
#         ai = np.array(c_jac_k,dtype ='float')[Ne:,:]

#         be = - np.array(ck,dtype ='float' )[:Ne,:]
#         bi = - np.array(ck,dtype ='float')[Ne:,:]

        G = np.array(L_hessian_xx_k).astype(float)
        c = np.array(f_jac_k).astype(float)
        ae = np.array(c_jac_k)[:Ne,:].astype(float)
        ai = np.array(c_jac_k)[Ne:,:].astype(float)

        be = - np.array(ck)[:Ne,:].astype(float)
        bi = - np.array(ck)[Ne:,:].astype(float)
        be[abs(be) < tol_zero] = 0.0    #  This line is ugly...but if the rhs is a small number like 1e-16, make the initial condition infeasible.
        p,lambda_k = QP_activeset(G, c, ae = ae, be = be, ai = ai, bi= bi, x0 = None)
        x_old = xk
        xk = xk + p
        if(np.linalg.norm(x_old - xk) < eps):
            return xk

### Test 1
$$
\begin{aligned}
\min _{x} q(x)  =\left(x_{1}\right)^{2}+& \left(x_{2}\right)^{2} + \left(x_{3}\right)^{2}\\
\text { subject to }  x_{1} + x_{2} + x_{2}  & = 1 \\
                      x_{1}   & \geq 1 
\end{aligned}
$$

In [20]:
x = sp.symbols('x0:3')
x_vec = sp.Matrix([x[0],x[1],x[2]])
P = np.diag([1,1,1])
obj_vec = x_vec.T @ np.diag([1,1,1]) @ x_vec
obj = obj_vec

ce = sp.Matrix([x[0]+x[1]+x[2]-1])
ci = sp.Matrix([x[0]])

x0 = np.array([1,0,0]).reshape(-1,1)

In [21]:
x_opt = SQP_solver(x_vec, obj,ce = ce, ci = ci,x0 =x0 )
x_opt

Solving with the local SQP algorithm
solution find
solution find


array([[0.33333333],
       [0.33333333],
       [0.33333333]])

### Test 2
$$
\begin{aligned}
\min _{x} q(x)  =\left(\sin(x_{1})\right)^{2}+& \left(x_{2}\right)^{2} + \left(x_{3}\right)^{2}\\
\text { subject to }  x_{1} + x_{2} + x_{2}  & = 1 \\
                      x_{1}   & \geq 0 \\ 
                      x_{1}   & \leq 1 
\end{aligned}
$$

In [22]:
x = sp.symbols('x0:3')
x_vec = sp.Matrix([x[0],x[1],x[2]])
obj_vec = sp.Matrix([sp.sin(x[0])**2+x[1]*x[1]+x[2]*x[2]])
obj = obj_vec

ce = sp.Matrix([x[0]+x[1]+x[2]-1])
ci = sp.Matrix([x[0],-x[0]+1])

x0 = np.array([0,1.5,-0.5]).reshape(-1,1)

In [23]:
x_opt = SQP_solver(x_vec, obj,ce = ce, ci = ci,x0 =x0 )
x_opt

Solving with the local SQP algorithm
solution find
solution find
solution find
solution find


array([[0.35228846],
       [0.32385577],
       [0.32385577]])

### Test 3
$$
\begin{aligned}
\min _{x} q(x)  =\left(\sin(x_{1})\right)+& \left(x_{2}\right)^{2} + \left(x_{3}\right)^{2}\\
\text { subject to }  x_{1} + x_{2} + x_{2}  & = 1 \\
                      x_{1}   & \geq 0 \\ 
                      x_{1}   & \leq 1 
\end{aligned}
$$

In [24]:
x = sp.symbols('x0:3')
x_vec = sp.Matrix([x[0],x[1],x[2]])
obj_vec = sp.Matrix([sp.sin(x[0])+x[1]*x[1]+x[2]*x[2]])
obj = obj_vec

ce = sp.Matrix([x[0]+x[1]+x[2]-1])
ci = sp.Matrix([x[0],-x[0]+1])

x0 = np.array([0,1.5,-0.5]).reshape(-1,1)

In [25]:
x_opt = SQP_solver(x_vec, obj,ce = ce, ci = ci,x0 =x0 )
x_opt

Solving with the local SQP algorithm
solution find
solution find


array([[0. ],
       [0.5],
       [0.5]])

### Line search SQP

### Trust region SQP